In [ ]:
import logging
import sys
logging.disable(sys.maxsize)

In [ ]:
from datetime import datetime

from pathlib import Path
import warnings
import os
import re
import pickle
import pandas as pd

import jax
import jax.numpy as jnp

from src.config.core import Config
from src.config.sampler import Sampler
from src.config.data import DatasetType
import src.dataset as ds
import src.training.utils as train_utils
import src.inference.utils as inf_utils
import src.visualization as viz
from src.config.data import Task
from src.inference.evaluation import evaluate_bde

from matplotlib import pyplot as plt
import numpy as np

In [ ]:
try:
    DIR = Path(os.environ['SANDBOX_EXPERIMENT_DIR'])
except KeyError as e:
    raise KeyError('Please set the "SANDBOX_EXPERIMENT_DIR" variable') from e

tree_dir = DIR / 'tree'

<h1><center>Log Summary</center></h1>

In [ ]:
def get_total_time(logs: str):
    log_lines = [line for line in logs.split('\n') if line]
    # Extract Timestamp
    if len(log_lines) < 2:
        return 0 
    start = re.findall(r'(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2})', log_lines[0])[0]
    end = re.findall(r'(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2})', log_lines[-1])[0]
    return (datetime.fromisoformat(end) - datetime.fromisoformat(start)).total_seconds()

with open(DIR / 'training.log', 'r') as f:
    logs = f.read()

warm_incomplete = False
bde_incomplete = False

warmstart_match = re.findall(r'time.warmstart took (\d+.\d+) seconds', logs)
bde_match = re.findall(r'time.sampling took (\d+.\d+) seconds', logs)
if warmstart_match:
    warmstart_time = float(warmstart_match[0])
    if bde_match:
        bde_time = float(bde_match[0])
    else:
        bde_time = get_total_time(logs) - warmstart_time
        bde_incomplete = True
else:
    warm_incomplete = True
    bde_incomplete = True
    warmstart_time = get_total_time(logs)
    bde_time = 0

total_time = round(warmstart_time + bde_time, 2)
print("-" * 50)
print(f'Warmstart time: {round(warmstart_time / 60, 2)} min{" (incomplete)" if warm_incomplete else ""}')
print(f'BDE time: {round(bde_time / 60, 2)} min{" (incomplete)" if bde_incomplete else ""}')
print(f'Total time: {round(total_time / 60, 2)} min{" (incomplete)" if warm_incomplete or bde_incomplete else ""}')
print("-" * 50)
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    config = Config.from_yaml(DIR / 'config.yaml')
# print(f"> Loaded Configuration:\n{config}")
metrics = {
    'total_time': total_time,
    'warmstart_time': warmstart_time,
    'bde_time': bde_time
}


<center><h1>Sampling</center></h1>

In [ ]:
samples = train_utils.load_samples_from_dir(DIR / 'samples', tree_path=tree_dir)
n_samples = inf_utils.count_samples(samples)
n_chains = inf_utils.count_chains(samples)
print(f'Loaded {n_chains} chains with {n_samples} samples each')
mean_ess = viz.plot_effective_sample_size(samples)

### Variance Analysis

In [ ]:
# mean_bcv, mean_wcv = viz.plot_variances(samples)
# mean_crhat = viz.plot_split_chain_r_hat(samples, n_splits=4)
# viz.plot_param_hist(samples, hist_limit=10000)

### Visualizing Sampling

In [ ]:
# viz.plot_pca(samples, '3d', max_chains=2, max_samples=None, annotate=True)
viz.plot_param_movement(samples, random_n=5)

In [ ]:
# # Combine the mean diagnostics into a dataframe and save it
# rows = list(mean_ess.keys())
# columns = ['ess', 'bcv', 'wcv', 'crhat']
# df = pd.DataFrame(index=rows, columns=columns)
# df['ess'] = mean_ess.values()
# df['bcv'] = mean_bcv.values()
# df['wcv'] = mean_wcv.values()
# df['crhat'] = mean_crhat.values()
# df.to_csv(DIR / 'diagnostics.csv')

<center><h1>BDE Performance</center></h1>

In [ ]:
# Extra arguments needed in forward pass of the flax model
kwargs = {}

# Setup Loader
if config.data.data_type == DatasetType.TABULAR:
    loader = ds.TabularLoader(
        config.data,
        rng=config.jax_rng,
        target_len=config.data.target_len
    )
elif config.data.data_type == DatasetType.TEXT:
    from src.dataset.utils import CustomBPETokenizer
    tokenizer = CustomBPETokenizer(
        config.model.context_len, max_vocab_size=config.model.vocab_size
    )
    loader = ds.TextLoader(
        config=config.data,
        rng=config.jax_rng,
        tokenizer=tokenizer,
        context_len=config.model.context_len,
        omit_freq=100,
    )
    kwargs = {'train': False, 'pad_id': tokenizer.padding_token_id}
elif config.data.data_type == DatasetType.IMAGE:
    loader = ds.ImageLoader(
        config.data,
        rng=config.jax_rng,
    )
else:
    raise NotImplementedError(f'DatasetType {config.data.data_type} not supported yet.')

random_input = next(loader.iter('train', 1))['feature']

print(f'> Loader:\n{loader}')
module = config.get_flax_model()

print('> Parameter Overview:\n')
print(module.tabulate(config.jax_rng, x=random_input, **kwargs))

with open(DIR / 'samples/info.pkl', 'rb') as f:
    info = pickle.load(f)

accept_rate = info.get("acceptance_rate", None)
integration_steps = info.get("num_integration_steps", None)
if accept_rate is not None:
    print("Average acceptance rate: ", accept_rate.mean())
    metrics['acceptance_rate'] = accept_rate.mean()
if integration_steps is not None:
    print("Average ingeration steps: ", integration_steps.mean())
    metrics['integration_steps'] = integration_steps.mean()

In [ ]:
features = loader.test_x # (B x F)
labels = loader.test_y # (B x T)
print("Test Set: Feature and Label have shapes: ", features.shape, labels.shape)

In [ ]:
# logits, metrics = evaluate_de(
#     params=params_warmstart,
#     module=module,
#     features=features,
#     labels=labels,
#     task=config.data.task,
#     batch_size=config.training.warmstart.batch_size,
#     verbose=False,
#     metrics_dict=metrics,
#     n_samples=config.training.sampler.n_samples // config.training.sampler.n_thinning,
#     rng_key=config.jax_rng,
#     nominal_coverages=[0.5, 0.75, 0.9, 0.95],
#     **kwargs
# )

In [ ]:
mem_cap = 4 * 1024 ** 3 # 4 GB
mem_usage = (
    inf_utils.get_mem_size(samples) / n_chains
    + inf_utils.get_mem_size(features)
    + inf_utils.get_mem_size(labels)
)
overhead_unit = mem_usage * 10 # estimatation

batch_size = max(1, (mem_cap - overhead_unit) // mem_usage)
if batch_size > labels.shape[0]:
    batch_size = None
print(f'> Batch size for evaluation: {batch_size}\n')

In [ ]:
logits, metrics = evaluate_bde(
    params=samples,
    module=module,
    features=features,
    labels=labels,
    task=config.data.task,
    batch_size=batch_size,
    verbose=True,
    metrics_dict=metrics,
    nominal_coverages=[0.5, 0.75, 0.9, 0.95],
    **kwargs
)

In [ ]:
# LPPD over the samples
viz.plot_lppd(logits, labels, config.data.task)

### Parallel vs. Sequential (NOT HERE)

In [ ]:
# with open(DIR / 'samples/info.pkl', 'rb') as f:
#     info = pickle.load(f)

# n_cycles = info['scheduler_config']['parameters']['n_cycles']
# cycle_length = n_samples // n_cycles

In [ ]:
# step_sizes = np.array(info['step_size'])
# explore = np.array(info['explore'])
# sampling_points = np.ma.masked_where(explore, step_sizes)
# fig, ax = plt.subplots()
# ax.plot(step_sizes, lw=2, ls="--", color="r", label="Exploration stage")
# ax.plot(sampling_points, lw=2, ls="-", color="k", label="Sampling stage")
# plt.show()

In [ ]:
# parallel = jax.tree.map(
#     lambda x: x[:, :cycle_length, :],
#     samples
# )

# print('Parallel')
# logits_par, metrics_par = evaluate_bde(
#     params=parallel,
#     module=module,
#     features=features,
#     labels=labels,
#     task=config.data.task,
#     batch_size=batch_size,
#     verbose=False,
#     metrics_dict=metrics.copy(),
#     nominal_coverages=[0.5, 0.75, 0.9, 0.95],
#     per_chain=False,
#     **kwargs
# )

In [ ]:
# logits_seq = []
# lppds = []
# rmses = []
# print('Sequential')
# for i in range(n_chains):
#     sequential = jax.tree.map(
#         lambda x: x[i, :, :][None, ...],
#         samples
#     )
#     print('=' * 50)
#     print(f'Chain {i}')
#     logits, metrics = evaluate_bde(
#         params=sequential,
#         module=module,
#         features=features,
#         labels=labels,
#         task=config.data.task,
#         batch_size=batch_size,
#         verbose=False,
#         metrics_dict=metrics,
#         nominal_coverages=[0.5, 0.75, 0.9, 0.95],
#         **kwargs
#     )
#     print('=' * 50 + '\n')
#     # logits_seq.append(logits)
#     # metrics_seq.append(metrics)
#     lppds.append(metrics['lppd'])
#     rmses.append(metrics['rmse'])

In [ ]:
# lppds = jnp.array(lppds)
# rmses = jnp.array(rmses)

# lppd_seq_mean = jnp.array(lppds).mean()
# rmse_seq_mean = jnp.array(rmses).mean()

# lppd_par = metrics_par['lppd']
# rmse_par = metrics_par['rmse']

# # print(f'Parallel LPPD: {lppd_par:.4f}, Sequential LPPD: {lppd_seq_mean:.4f}')
# # print(f'Parallel RMSE: {rmse_par:.4f}, Sequential RMSE: {rmse_seq_mean:.4f}')

# # Summary table
# data = {
#     "Chain": [f"Chain {i}" for i in range(len(lppds))] + ["Sequential Mean", "Parallel"],
#     "LPPD": list(lppds) + [lppd_seq_mean, lppd_par],
#     "RMSE": list(rmses) + [rmse_seq_mean, rmse_par],
# }
# df = pd.DataFrame(data)
# pickle.dump(df, open(DIR / 'summary.pkl', 'wb'))
# df.style.format({
#     "LPPD": "{:.4f}",
#     "RMSE": "{:.4f}"
# })

### Training log

In [ ]:
df = pd.read_csv(DIR / "metrics.csv")

fig, axs = plt.subplots(figsize=(8, 8), nrows=2)
for i in range(n_chains):
    chain_df = df[df["chain"] == i]
    axs[0].plot(chain_df["step_count"], chain_df["log_posterior"], label=f"Chain {i}")
    axs[1].plot(chain_df["step_count"], chain_df["rmse"], label=f"Chain {i}")
axs[0].set_title("Log Posterior")
axs[1].set_title("RMSE")
plt.show()